In [ ]:
!pip install transformers pandas numpy scikit-learn tensorflow -q

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import os

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
def load_and_validate_data(file_path="/content/drive/My Drive/Project and Coding/BERT AND ELECTRA/Apple_tweets_setiment.csv"):
    try:
        df = pd.read_csv(file_path, on_bad_lines='skip', quoting=3)
        if 'tweets' not in df.columns or 'labels' not in df.columns:
            raise ValueError("Dataset must contain 'tweets' and 'labels' columns")
        df = df.dropna(subset=['tweets', 'labels'])

        # Filter out neutral (0.0) labels. We only do for binary sentiment
        df = df[df['labels'] != 0.0]

        df['labels'] = df['labels'].apply(lambda x: 0 if x == -1.0 else 1)

        print("Dataset Info:")
        print(df.info())
        print("\nSample Data:")
        print(df.head())
        print("\nLabel Distribution:")
        print(df['labels'].value_counts())

        return df

    except FileNotFoundError:
        print("Error: Dataset file not found.")
        return None


In [ ]:
df=load_and_validate_data()
if df is None:
  raise SystemExit("Failed to load dataset. Exiting.")

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 2149 entries, 5 to 4532
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  2149 non-null   int64 
 1   tweets  2149 non-null   object
dtypes: int64(1), object(1)
memory usage: 50.4+ KB
None

Sample Data:
    labels                                             tweets
5        1            top 3 all @apple #tablets. damn right! 
6        1  cnbctv: #apple's margins better than expected?...
9        0  wtf my battery was 31% one second ago and now ...
11       1  rt @peterpham: bought my @augustsmartlock at t...
12       0  @apple contact sync between yosemite and ios8 ...

Label Distribution:
labels
0    1648
1     501
Name: count, dtype: int64


In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['tweets'],
    df['labels'],
    test_size=0.2,
    random_state=42,
    stratify=df['labels']
)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Tokenize data
def tokenize_data(texts, max_length=128):
    return tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='tf'
    )

train_encodings = tokenize_data(train_texts)
test_encodings = tokenize_data(test_texts)

In [ ]:
def create_dataset(encodings, labels):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
            'token_type_ids': encodings['token_type_ids']
        },
        labels
    ))
    return dataset

train_dataset = create_dataset(train_encodings, tf.convert_to_tensor(list(train_labels)))
test_dataset = create_dataset(test_encodings, tf.convert_to_tensor(list(test_labels)))

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy('accuracy'),
    tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='top_1_accuracy')
]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=1,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/Project and Coding/BERT AND ELECTRA/best_bert_model_2',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )
]

In [ ]:
print("\nTraining BERT model...")
history = model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=3,
    batch_size=16,
    validation_data=test_dataset.batch(16),
    callbacks=callbacks
)


Training BERT model...
Epoch 1/3
108/108 [==============================] - 118s 678ms/step - loss: 0.3800 - accuracy: 0.8278 - top_1_accuracy: 0.8278 - val_loss: 0.1775 - val_accuracy: 0.9395 - val_top_1_accuracy: 0.9395
Epoch 2/3
108/108 [==============================] - 65s 601ms/step - loss: 0.1431 - accuracy: 0.9552 - top_1_accuracy: 0.9552 - val_loss: 0.1526 - val_accuracy: 0.9488 - val_top_1_accuracy: 0.9488
Epoch 3/3
108/108 [==============================] - 101s 943ms/step - loss: 0.0605 - accuracy: 0.9820 - top_1_accuracy: 0.9820 - val_loss: 0.1672 - val_accuracy: 0.9535 - val_top_1_accuracy: 0.9535


In [ ]:
print("\nEvaluating model...")
predictions = model.predict(test_dataset.batch(16))
pred_labels = np.argmax(predictions.logits, axis=1)


Evaluating model...
27/27 [==============================] - 5s 69ms/step


In [ ]:
print("\nClassification Report:")
print(classification_report(test_labels, pred_labels, target_names=['Negative', 'Positive']))
print("Accuracy:", accuracy_score(test_labels, pred_labels))
print("F1 Score:", f1_score(test_labels, pred_labels))


Classification Report:
              precision    recall  f1-score   support

    Negative       0.96      0.98      0.97       330
    Positive       0.92      0.85      0.89       100

    accuracy                           0.95       430
   macro avg       0.94      0.91      0.93       430
weighted avg       0.95      0.95      0.95       430

Accuracy: 0.9488372093023256
F1 Score: 0.8854166666666666


In [ ]:
save_path = "/content/drive/MyDrive/Project and Coding/BERT AND ELECTRA/bert_sentiment_model_2"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"\nModel saved to {save_path}")


Model saved to /content/drive/MyDrive/Project and Coding/BERT AND ELECTRA/bert_sentiment_model_2


In [ ]:
# Prediction function
def predict_sentiment(text):
    try:
        inputs = tokenizer(
            text,
            return_tensors="tf",
            truncation=True,
            padding=True,
            max_length=128
        )
        outputs = model(inputs)
        probs = tf.nn.softmax(outputs.logits, axis=-1)
        pred = np.argmax(probs, axis=1)[0]
        confidence = probs[0][pred].numpy()
        return ("Positive" if pred == 1 else "Negative", confidence)
    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        return None, None

In [ ]:
sample_texts = [
    "Absolutely love the new iPhone! Apple nailed it again.",
    "Apple stock is dropping after the disappointing earnings report.",
    "Thinking of switching to Android, tired of Apple's constant price hikes.",
    "The new iOS update is so smooth and fast. Great job Apple!",
    "Apple's customer service was incredibly helpful today.",
    "My MacBook just crashed again, so frustrating!",
    "Apple Pay makes checkout so much easier. Love this feature!",
    "Battery life on my iPhone is terrible after the latest update.",
    "AirPods Pro sound amazing! Totally worth the price.",
    "Apple Watch saved my life by detecting my irregular heartbeat."
]

print("\nExample Predictions:")
for text in sample_texts:
    sentiment, confidence = predict_sentiment(text)
    print(f"Text: {text}")
    if sentiment is not None and confidence is not None:
        print(f"Sentiment: {sentiment}, Confidence: {confidence:.4f}\n")
    else:
        print(f"Error predicting sentiment for text: {text}\n")


Example Predictions:
Text: Absolutely love the new iPhone! Apple nailed it again.
Sentiment: Positive, Confidence: 0.9843

Text: Apple stock is dropping after the disappointing earnings report.
Sentiment: Negative, Confidence: 0.9588

Text: Thinking of switching to Android, tired of Apple's constant price hikes.
Sentiment: Negative, Confidence: 0.9713

Text: The new iOS update is so smooth and fast. Great job Apple!
Sentiment: Positive, Confidence: 0.9816

Text: Apple's customer service was incredibly helpful today.
Sentiment: Positive, Confidence: 0.9762

Text: My MacBook just crashed again, so frustrating!
Sentiment: Negative, Confidence: 0.9666

Text: Apple Pay makes checkout so much easier. Love this feature!
Sentiment: Positive, Confidence: 0.9788

Text: Battery life on my iPhone is terrible after the latest update.
Sentiment: Negative, Confidence: 0.9855

Text: AirPods Pro sound amazing! Totally worth the price.
Sentiment: Positive, Confidence: 0.9331

Text: Apple Watch saved my